<a href="https://colab.research.google.com/github/kimhwijin/HandsOnMachineLearing/blob/main/HOML_Exercise_16.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl


In [25]:
np.random.seed(42)
tf.random.set_seed(42)

#8

In [26]:
#문법에 맞는 문자열을 반환하는 함수

#문법, LR table 과 비슷함
default_reber_grammar = [
    [("B", 1)],           # (state 0) =B=>(state 1)
    [("T", 2), ("P", 3)], # (state 1) =T=>(state 2) or =P=>(state 3)
    [("S", 2), ("X", 4)], # (state 2) =S=>(state 2) or =X=>(state 4)
    [("T", 3), ("V", 5)], # and so on...
    [("X", 3), ("S", 6)],
    [("P", 4), ("V", 6)],
    #마지막
    [("E", None)]]        # (state 6) =E=>(terminal state)

embedded_reber_grammar = [
    [("B", 1)],
    [("T", 2), ("P", 3)],
    [(default_reber_grammar, 4)],
    [(default_reber_grammar, 5)],
    [("T", 6)],
    [("P", 6)],
    #마지막
    [("E", None)]]

def generate_string(grammar):
    state = 0
    output=[]
    while state is not None:
        index = np.random.randint(len(grammar[state]))
        #다음 문자와 상태
        production, state = grammar[state][index]
        if isinstance(production, list):
            production = generate_string(grammar=production)
        output.append(production)
    return "".join(output)

In [28]:
for _ in range(25):
    #25개의 문자열 생성
    print(generate_string(default_reber_grammar), end=" ")

BTXSE BPTTTTTTTTTVPSE BPVPXTTVPXTTTVPSE BTXXTVVE BPTVPXVVE BTSXSE BPTTTTTTVPXTVPXVPXTTTVVE BPTTTVPXVPSE BPVPXTTTTTVPSE BTSSXSE BPVPSE BTSXSE BPVVE BPVVE BTXSE BTXXTVPXTTVPSE BTSSSXSE BTSXXTTVVE BPTTVVE BPVPSE BTXXVVE BPVPSE BTSSSXXTTVVE BPVPXVPXTVVE BTXSE 

In [35]:
#모든 가능한 알파벳들
POSSIBLE_CHARS = "BEPSTVX"

#잘못된 문자열 만들기
def generate_corrupted_string(grammar, chars=POSSIBLE_CHARS):
    #올바른 문자열에
    good_string = generate_string(grammar)
    index = np.random.randint(len(good_string))
    good_char = good_string[index]
    #있을수는 있지만 올바른 문자열엔 없는 문자열을
    bad_char = np.random.choice(sorted(set(chars) - set(good_char)))
    #중간에 하나 추가함
    return good_string[:index] + bad_char + good_string[index + 1:]

In [36]:
#잘못된 문자열
for _ in range(25):
    print(generate_corrupted_string(embedded_reber_grammar), end=" ")

BTBTXXVVVTE STBTSSXSETE BPBTPSXXVVEPE BXBTXXTTTVVETE BSBTXSETE BTBPTVPXBTVVETE BPBPTBVEPE BTBPVPTVPXVPXVPSETE BPBTXXTVVPPE BPBPVPXVPBEPE BTPTXSETE BVBPVVEPE BTBPVPXVPXVXETE BSBPVVEPE BSBTXXVVEPE ETBTSSSXXTTVVETE BPBXVPXTVPSEPE BTBPTSVVETE BPBTSSXSEPP BTBPVPXTTVBETE BPBPTETVVEPE BTVTXSETE BBBPTTTVVEPE BPBPTTVSSEPE BTBTSSXXVVVTE 

In [44]:
#문자열을 숫자로 바꾸기, 문자의 인덱스 리스트로 변경한다.
def strings_to_ids(s, chars=POSSIBLE_CHARS):
    return [chars.index(c) for c in s]

In [45]:
#올바른 문자열, 잘못된 문자열 1 : 1 인 샘플과 레이블 생성
def generate_dataset(size):
    good_strings = [strings_to_ids(generate_string(embedded_reber_grammar)) for _ in range(size // 2)]
    bad_strings = [strings_to_ids(generate_corrupted_string(embedded_reber_grammar)) for _ in range(size // 2)]
    all_strings = good_strings + bad_strings
    #ragged 텐서 생성
    X = tf.ragged.constant(all_strings, ragged_rank=1)
    y = np.array([[1.] for _ in range(len(good_strings))] + [[0.] for _ in range(len(bad_strings))])
    return X, y


In [46]:
X_train, y_train = generate_dataset(10000)
X_test, y_test = generate_dataset(2000)

In [47]:
X_train[0]

<tf.Tensor: shape=(11,), dtype=int32, numpy=array([0, 4, 0, 2, 4, 4, 5, 5, 1, 4, 1], dtype=int32)>